In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
from nltk.stem import SnowballStemmer, PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing,linear_model, naive_bayes, metrics, svm, tree, neighbors
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from io import StringIO
import seaborn as sns
import sklearn.feature_extraction.text as text
import os
from textblob import TextBlob
from textblob import Word
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
data.drop(['id'],axis=1, inplace=True)

In [5]:
data.drop(['keyword', 'location'],axis=1, inplace=True)

In [6]:
data.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7613 non-null   object
 1   target  7613 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 119.1+ KB


In [8]:
data.describe()

,target
count,7613.00000
mean,0.42966
std,0.49506
min,0.00000
25%,0.00000
50%,0.00000
75%,1.00000
max,1.00000


In [9]:
data['text'] = data['text'].apply(lambda x:" ".join(x.lower() for x in x.split()))

In [10]:
stop = stopwords.words('english')
data['text'] = data['text'].apply(lambda x:" ".join(x for x in x.split() if x not in stop))

In [11]:
st = PorterStemmer()
data['text'] = data['text'].apply(lambda x:" ".join(st.stem(word) for word in x.split()))

In [12]:
data['text'] = data['text'].apply(lambda x:" ".join(Word(word).lemmatize() for word in x.split()))

In [13]:
data.head()

,text,target
0,deed reason #earthquak may allah forgiv u,1
1,forest fire near la rong sask. canada,1
2,resid ask 'shelter place' notifi officers. eva...,1
3,"13,000 peopl receiv #wildfir evacu order calif...",1
4,got sent photo rubi #alaska smoke #wildfir pou...,1


In [14]:
train_x, valid_x, train_y, valid_y = train_test_split(data['text'], data['target'])

In [16]:
tfid_vec = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfid_vec.fit(data['text'])

TfidfVectorizer(max_features=5000, token_pattern='\\w{1,}')

In [17]:
xtrain_tfidf = tfid_vec.transform(train_x)
xvalid_tfidf = tfid_vec.transform(valid_x)

In [ ]:
tfid_vec = CountVectorizer(ngram_range=(1,2))
tfid_vec.fit(data['text'])
xtrain_tfidf = tfid_vec.transform(train_x)
xvalid_tfidf = tfid_vec.transform(valid_x)

In [36]:
#def train_model(classifier, feature_vector_train, label,feature_vector_valid, is_neural_net=False):
classifier = linear_model.LogisticRegression()
classifier.fit(xtrain_tfidf, train_y)
predictions = classifier.predict(xvalid_tfidf)
accuracies = metrics.accuracy_score(predictions, valid_y)
confusion = metrics.r2_score(predictions, valid_y)


In [26]:
accuracy,confusion = train_model(naive_bayes.MultinomialNB(alpha=0.2), xtrain_tfidf, train_y, xvalid_tfidf)
print('Accuracy: ', accuracy)
print('R2score: ', confusion)

Accuracy:  0.8051470588235294
R2score:  0.16433434481453202


In [29]:
accuracy1,confusion1 = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print('Accuracy: ', accuracy1)
print('R2score: ', confusion1)

Accuracy:  0.8103991596638656
R2score:  0.16461083906692286


In [21]:
accuracy,confusion = train_model(svm.LinearSVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print('Accuracy: ', accuracy)
print('R2score: ', confusion)

Accuracy:  0.7878151260504201
R2score:  0.11124667822068202


In [22]:
accuracy,confusion = train_model(neighbors.KNeighborsClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print('Accuracy: ', accuracy)
print('R2score: ', confusion)

Accuracy:  0.6544117647058824
R2score:  -2.2539400550620745


In [23]:
accuracy,confusion = train_model(tree.ExtraTreeClassifier(max_features=4000), xtrain_tfidf, train_y, xvalid_tfidf)
print('Accuracy: ', accuracy)
print('R2score: ', confusion)

Accuracy:  0.7247899159663865
R2score:  -0.1465081750945756


In [27]:
import pickle

In [37]:
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(classifier, file)

In [38]:
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

In [39]:
pickle_model.coef_

array([[ 0.28360163,  0.39440484, -0.20358658, ...,  0.02147353,
         0.2810965 , -0.03058492]])

In [51]:
cv = CountVectorizer(ngram_range=(1,2),max_features=5000)
nf = cv.fit_transform(data1['text'])

In [52]:
pred = pickle_model.predict(nf)

In [53]:
pred.reshape(len(pred))

array([1, 0, 0, ..., 1, 1, 0], dtype=int64)

In [41]:
data1 = pd.read_csv('test.csv')

In [42]:
data1

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [43]:
data1.drop(['keyword', 'location'],axis=1, inplace=True)

In [44]:
data1.drop(['id'],axis=1, inplace=True)

In [45]:
data1.head()

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan
